## Hard Constraint Evaluation Function

This function evaluates a timetable to check for violations of hard constraints — rules that must not be broken for the schedule to be valid. It counts different types of violations and prints a summary, helping us understand where the schedule fails.

### Parameters:
- **`timetable`:** A dictionary where each timeslot contains rooms and the activities scheduled in them.
- **`activities_dict`:** A dictionary of all activities, indexed by activity ID.
- **`groups_dict`:** A dictionary of student groups, including their sizes.
- **`spaces_dict`:** A dictionary of rooms, including their capacities.

### What the function checks:

1. **Vacant Rooms:**  
   If a room is left empty in a timeslot, it’s counted as a vacant room. 

2. **Lecturer Conflicts:**  
   If the same lecturer is assigned to more than one activity in the same timeslot, that counts as a conflict.

3. **Student Group Conflicts:**  
   If a student group is scheduled for multiple activities in the same timeslot, it creates a conflict. The function uses set intersections to catch these overlaps.

4. **Room Capacity Violations:**  
   The function checks if the total number of students in an activity exceeds the room’s capacity. If it does, that counts as a violation.

5. **Unassigned Activities:**  
   The function also counts how many activities were never assigned to any room or timeslot.

### The output:  
After checking all the constraints, the function prints a summary:  
- **Vacant Rooms Count** — how many rooms were left empty.  
- **Lecturer Conflict Violations** — how many times lecturers were double-booked.  
- **Student Group Conflict Violations** — how many times student groups were double-booked.  
- **Room Capacity Violations** — how many times room size was insufficient for the assigned activity.  
- **Unassigned Activity Violations** — how many activities weren’t placed in the timetable.  

Finally, it adds up all the violations to give a **total hard constraint violation score**. This score gives a quick sense of how well the timetable satisfies the strictest rules. Lower scores are better, meaning fewer conflicts and a more feasible schedule.

This function is essential for checking the raw feasibility of a schedule before worrying about things like preferences or soft constraints. 


In [1]:

def evaluate_hard_constraints(timetable, activities_dict, groups_dict, spaces_dict):
    vacant_rooms = []
    vacant_room = 0
    prof_conflicts = 0
    room_size_conflicts = 0
    sub_group_conflicts = 0
    unasigned_activities = len(activities_dict)
    activities_set = set()

    for slot in timetable:
        prof_set = set()
        sub_group_set = set()
        for room in timetable[slot]:
            activity = timetable[slot][room]

            if not isinstance(activity, type(list(activities_dict.values())[0])):  # Ensure it's an Activity object
                vacant_room += 1
                vacant_rooms.append((slot, room))
            else:
                activities_set.add(activity.id)

                # Lecturer Conflict Check
                if activity.teacher_id in prof_set:
                    prof_conflicts += 1
                prof_set.add(activity.teacher_id)

                # Student Group Conflict Check
                sub_group_conflicts += len(
                    set(activity.group_ids).intersection(sub_group_set))

                group_size = 0
                for group_id in activity.group_ids:
                    group_size += groups_dict[group_id].size
                    sub_group_set.add(group_id)

                # Room Capacity Constraint Check
                if group_size > spaces_dict[room].size:
                    room_size_conflicts += 1

    # Unassigned Activity Count
    unasigned_activities -= len(activities_set)

    # Print Results
    print("\n--- Hard Constraint Evaluation Results ---")
    print(f"Vacant Rooms Count: {vacant_room}")
    print(f"Lecturer Conflict Violations: {prof_conflicts}")
    print(f"Student Group Conflict Violations: {sub_group_conflicts}")
    print(f"Room Capacity Violations: {room_size_conflicts}")
    print(f"Unassigned Activity Violations: {unasigned_activities}")

    # Final Hard Constraint Violation Score
    total_violations = prof_conflicts + sub_group_conflicts + room_size_conflicts + unasigned_activities
    print(f"\nTotal Hard Constraint Violations: {total_violations}")


## Soft Constraint Evaluation Function

This function evaluates the **soft constraints** of a timetable — factors that influence schedule quality but can be compromised if necessary. It measures aspects like student and lecturer fatigue, idle time, spread of lectures, and lecturer workload balance. The function then computes an overall score to help us understand how well the schedule performs.

### Parameters:
- **`schedule`:** A dictionary representing the scheduled activities, organized by time slots and room assignments.
- **`groups_dict`:** A dictionary containing student group details (e.g., group size).
- **`lecturers_dict`:** A dictionary containing lecturer details.
- **`slots`:** An ordered list of available time slots.

### What the function checks:

1. **Student Metrics:**  
   - **Fatigue:** Number of lectures attended.  
   - **Idle Time:** Gaps between lectures within the same day.  
   - **Lecture Spread:** Distribution of lectures across slots (more spread = more scattered, less compact).

2. **Lecturer Metrics:**  
   - **Fatigue:** Number of lectures conducted.  
   - **Idle Time:** Gaps between lectures.  
   - **Lecture Spread:** How scattered the lectures are across the slots.  
   - **Workload Balance:** Variance in workload across lecturers (lower variance = better balance).

### How the function works:

- It loops through each timeslot and room to gather relevant data on activities.  
- It updates fatigue, spread, and workload metrics directly during this loop.  
- It calculates idle time by checking gaps between consecutive lectures.  
- It normalizes all metrics for fair comparison and calculates workload balance using variance.  

### Scoring the constraints:

The function prints individual scores for:  
- **Student Fatigue Factor**  
- **Student Idle Time Factor**  
- **Student Lecture Spread Factor**  
- **Lecturer Fatigue Factor**  
- **Lecturer Idle Time Factor**  
- **Lecturer Lecture Spread Factor**  
- **Lecturer Workload Balance Factor**  

Finally, it computes a **weighted final score**. Higher scores indicate better schedule quality, with a balance between minimizing fatigue, idle time, and spread, while maximizing workload balance for lecturers.

The weights reflect the relative importance of each factor, but these can be adjusted as needed.

This function is invaluable for refining a feasible schedule into an optimized one that enhances the well-being of both students and lecturers.


In [2]:
import numpy as np

def evaluate_soft_constraints(schedule, groups_dict, lecturers_dict, slots):
    """
    Evaluates the soft constraints of a given schedule, measuring factors such as:
    - Student fatigue, idle time, and lecture spread across different slots.
    - Lecturer fatigue, idle time, lecture spread, and workload balance.

    Parameters:
    - schedule (dict): The scheduled activities mapped by time slots and locations.
    - groups_dict (dict): Dictionary of student groups with group IDs as keys.
    - lecturers_dict (dict): Dictionary of lecturers with lecturer IDs as keys.
    - slots (list): Ordered list of available time slots.

    Returns:
    - final_score (float): The computed soft constraint score representing 
      schedule quality based on fatigue, idle time, spread, and workload balance.
    """

    # Initialize student group metrics
    group_fatigue = {g: 0 for g in groups_dict.keys()}
    group_idle_time = {g: 0 for g in groups_dict.keys()}
    group_lecture_spread = {g: 0 for g in groups_dict.keys()}

    # Initialize lecturer metrics
    lecturer_fatigue = {l: 0 for l in lecturers_dict.keys()}
    lecturer_idle_time = {l: 0 for l in lecturers_dict.keys()}
    lecturer_lecture_spread = {l: 0 for l in lecturers_dict.keys()}
    lecturer_workload = {l: 0 for l in lecturers_dict.keys()}

    # Track the lecture slots assigned to each group and lecturer
    group_lecture_slots = {g: [] for g in groups_dict.keys()}
    lecturer_lecture_slots = {l: [] for l in lecturers_dict.keys()}

    # Process the schedule and accumulate lecture-related data
    for slot, rooms in schedule.items():
        for room, activity in rooms.items():
            # Process student groups
            for group_id in activity.group_ids:
                if group_id in groups_dict:
                    group_fatigue[group_id] += 1  # Increase fatigue per lecture
                    group_lecture_spread[group_id] += 2  # Increase spread factor
                    group_lecture_slots[group_id].append(slot)  # Store time slot

            # Process lecturers
            lecturer_id = activity.teacher_id
            if lecturer_id in lecturers_dict:
                lecturer_fatigue[lecturer_id] += 1
                lecturer_lecture_spread[lecturer_id] += 2
                lecturer_workload[lecturer_id] += activity.duration  # Add workload
                lecturer_lecture_slots[lecturer_id].append(slot)  # Store time slot

    # Compute idle time for each student group
    for group_id, lectures in group_lecture_slots.items():
        if lectures:
            lecture_indices = sorted([slots.index(s) for s in lectures])
            idle_time = sum(
                (lecture_indices[i + 1] - lecture_indices[i] - 1) for i in range(len(lecture_indices) - 1)
            )
            group_idle_time[group_id] = idle_time / (len(slots) - 1)  # Normalize

    # Compute idle time for each lecturer
    for lecturer_id, lectures in lecturer_lecture_slots.items():
        if lectures:
            lecture_indices = sorted([slots.index(s) for s in lectures])
            idle_time = sum(
                (lecture_indices[i + 1] - lecture_indices[i] - 1) for i in range(len(lecture_indices) - 1)
            )
            lecturer_idle_time[lecturer_id] = idle_time / (len(slots) - 1)  # Normalize

    # Helper function to normalize values within a dictionary
    def normalize(dictionary):
        max_val = max(dictionary.values(), default=1)
        return {k: v / max_val if max_val else 0 for k, v in dictionary.items()}

    # Normalize metrics for fair comparison
    group_fatigue = normalize(group_fatigue)
    group_idle_time = normalize(group_idle_time)
    group_lecture_spread = normalize(group_lecture_spread)
    lecturer_fatigue = normalize(lecturer_fatigue)
    lecturer_idle_time = normalize(lecturer_idle_time)
    lecturer_lecture_spread = normalize(lecturer_lecture_spread)

    # Compute lecturer workload balance
    workload_values = np.array(list(lecturer_workload.values()))
    lecturer_workload_balance = 1  # Default balance
    if len(workload_values) > 1 and np.mean(workload_values) != 0:
        lecturer_workload_balance = max(0, 1 - (np.var(workload_values) / np.mean(workload_values)))

    # Compute the final soft constraint metrics
    student_fatigue_score = np.mean(list(group_fatigue.values()))
    student_idle_time_score = np.mean(list(group_idle_time.values()))
    student_lecture_spread_score = np.mean(list(group_lecture_spread.values()))

    lecturer_fatigue_score = np.mean(list(lecturer_fatigue.values()))
    lecturer_idle_time_score = np.mean(list(lecturer_idle_time.values()))
    lecturer_lecture_spread_score = np.mean(list(lecturer_lecture_spread.values()))

    # Print individual final metric scores
    print("\n--- Soft Constraint Evaluation Results ---")
    print(f"Student Fatigue Factor: {student_fatigue_score:.2f}")
    print(f"Student Idle Time Factor: {student_idle_time_score:.2f}")
    print(f"Student Lecture Spread Factor: {student_lecture_spread_score:.2f}")
    print(f"Lecturer Fatigue Factor: {lecturer_fatigue_score:.2f}")
    print(f"Lecturer Idle Time Factor: {lecturer_idle_time_score:.2f}")
    print(f"Lecturer Lecture Spread Factor: {lecturer_lecture_spread_score:.2f}")
    print(f"Lecturer Workload Balance Factor: {lecturer_workload_balance:.2f}")

    # Compute final soft constraint score based on weighted factors
    final_score = (
        student_fatigue_score * 0.2 +
        (1 - student_idle_time_score) * 0.2 +
        (1 - student_lecture_spread_score) * 0.2 +
        (1 - lecturer_fatigue_score) * 0.1 +
        (1 - lecturer_idle_time_score) * 0.1 +
        (1 - lecturer_lecture_spread_score) * 0.1 +
        lecturer_workload_balance * 0.1
    )

    print(f"\nFinal Soft Constraint Score: {final_score:.2f}")
    return final_score


### Constraint Evaluation Function

This function evaluates a schedule by checking both **hard** and **soft constraints**:  

- **Hard Constraints:**  
  - Room availability and capacity.  
  - Lecturer and student group conflicts.  
  - Unassigned activities.  

- **Soft Constraints:**  
  - Student fatigue, idle time, and lecture spread.  
  - Lecturer fatigue, idle time, spread, and workload balance.  

By running both evaluations, we get a complete view of schedule feasibility and quality, helping us identify and fix violations while optimizing for better resource usage and well-being.  

In [3]:
# Constraint Evaluation Metrics
def evaluate(schedule, groups_dict, lecturers_dict, activities_dict, spaces_dict, slots):
    # Evaluate Hard Constraints
    evaluate_hard_constraints(schedule, activities_dict, groups_dict, spaces_dict)

    # Evaluate Soft Constraints
    evaluate_soft_constraints(schedule, groups_dict, lecturers_dict, slots)

# Timetable Data Processing

This script processes timetable data from a JSON file, structuring it using classes for spaces, groups, activities, periods, and lecturers.

### Class Definitions

- **Space**: Represents a location with a unique code and capacity.  
- **Group**: Defines a student group with an ID and size.  
- **Activity**: Represents a subject, assigned teacher, student groups, and duration.  
- **Period**: Associates an activity with a time slot and space.  
- **Lecturer**: Stores lecturer details, including ID, name, username, and department.

### Data Loading and Processing

The script reads `sliit_computing_dataset.json` and organizes data into:

- `spaces_dict`: Maps spaces by code.  
- `groups_dict`: Maps student groups by ID.  
- `activities_dict`: Maps activities by code.  
- `lecturers_dict`: Stores lecturers filtered by role.  

Each section is processed by iterating over the JSON file and creating class instances.

### Time Slot Generation

A list of time slots is created for Monday to Friday, with eight slots per day.

### Data Verification

The script prints the structured dictionaries to confirm correct data loading, providing a foundation for scheduling and further analysis.


In [4]:
class Space:
    def __init__(self, *args):
        self.code = args[0]
        self.size = args[1]

    def __repr__(self):
        return f"Space(code={self.code}, size={self.size})"


class Group:
    def __init__(self, *args):
        self.id = args[0]
        self.size = args[1]

    def __repr__(self):
        return f"Group(id={self.id}, size={self.size})"


class Activity:
    def __init__(self, id, *args):
        self.id = id
        self.subject = args[0]
        self.teacher_id = args[1]
        self.group_ids = args[2]
        self.duration = args[3]

    def __repr__(self):
        return f"Activity(id={self.id}, subject={self.subject}, teacher_id={self.teacher_id}, group_ids={self.group_ids}, duration={self.duration})"


class Period:
    def __init__(self, *args):
        self.space = args[0]
        self.slot = args[1]
        self.activity = args[2]

    def __repr__(self):
        return f"Period(space={self.space}, group={self.group}, activity={self.activity})"

class Lecturer:
    def __init__(self, id, first_name, last_name, username, department):
        self.id = id
        self.first_name = first_name
        self.last_name = last_name
        self.username = username
        self.department = department

    def __repr__(self):
        return f"Lecturer(id={self.id}, name={self.first_name} {self.last_name}, department={self.department})"



import json

# Load data from JSON file
with open('sliit_computing_dataset.json', 'r') as file:
    data = json.load(file)

# Create dictionaries to store instances
spaces_dict = {}
groups_dict = {}
activities_dict = {}
lecturers_dict = {}
slots = []
# Populate the dictionaries with data from the JSON file
for space in data['spaces']:
    spaces_dict[space['code']] = Space(space['code'], space['capacity'])

for group in data['years']:
    groups_dict[group['id']] = Group(group['id'], group['size'])

for activity in data['activities']:
    activities_dict[activity['code']] = Activity(
        activity['code'], activity['subject'], activity['teacher_ids'][0], activity['subgroup_ids'], activity['duration'])

for user in data["users"]:
    if user["role"] == "lecturer":
        lecturers_dict[user["id"]] = Lecturer(
            user["id"], user["first_name"], user["last_name"], user["username"], user["department"]
        )

for day in ["MON", "TUE", "WED", "THU", "FRI"]:
    for id in range(1, 9):
        slots.append(day+str(id))
# Print the dictionaries to verify
print("spaces_dict=", spaces_dict)
print("groups_dict=", groups_dict)
print("activities_dict=", activities_dict)
print("lecturers_dict=", lecturers_dict)
print("slots=",slots)

spaces_dict= {'LH401': Space(code=LH401, size=200), 'LH501': Space(code=LH501, size=200), 'LAB501': Space(code=LAB501, size=60), 'LAB502': Space(code=LAB502, size=60)}
groups_dict= {'Y1S1.1': Group(id=Y1S1.1, size=40), 'Y1S1.2': Group(id=Y1S1.2, size=40), 'Y1S1.3': Group(id=Y1S1.3, size=40), 'Y1S1.4': Group(id=Y1S1.4, size=40), 'Y1S1.5': Group(id=Y1S1.5, size=40), 'Y1S2.1': Group(id=Y1S2.1, size=40), 'Y1S2.2': Group(id=Y1S2.2, size=40), 'Y1S2.3': Group(id=Y1S2.3, size=40), 'Y1S2.4': Group(id=Y1S2.4, size=40), 'Y1S2.5': Group(id=Y1S2.5, size=40), 'Y2S1.1': Group(id=Y2S1.1, size=40), 'Y2S1.2': Group(id=Y2S1.2, size=40), 'Y2S1.3': Group(id=Y2S1.3, size=40), 'Y2S1.4': Group(id=Y2S1.4, size=40), 'Y2S1.5': Group(id=Y2S1.5, size=40), 'Y2S2.1': Group(id=Y2S2.1, size=40), 'Y2S2.2': Group(id=Y2S2.2, size=40), 'Y2S2.3': Group(id=Y2S2.3, size=40), 'Y2S2.4': Group(id=Y2S2.4, size=40), 'Y2S2.5': Group(id=Y2S2.5, size=40), 'Y3S1.1': Group(id=Y3S1.1, size=40), 'Y3S1.2': Group(id=Y3S1.2, size=40), 'Y3S

In [5]:
class Period:
    def __init__(self, space, slot, activity=None):
        self.space = space
        self.slot = slot
        self.activity = activity

    def __repr__(self):
        return f"Period(space={self.space}, slot={self.slot}, activity={self.activity})"

slots = ['MON1', 'MON2', 'MON3', 'MON4', 'MON5', 'MON6', 'MON7', 'MON8',
         'TUE1', 'TUE2', 'TUE3', 'TUE4', 'TUE5', 'TUE6', 'TUE7', 'TUE8',
         'WED1', 'WED2', 'WED3', 'WED4', 'WED5', 'WED6', 'WED7', 'WED8',
         'THU1', 'THU2', 'THU3', 'THU4', 'THU5', 'THU6', 'THU7', 'THU8',
         'FRI1', 'FRI2', 'FRI3', 'FRI4', 'FRI5', 'FRI6', 'FRI7', 'FRI8']

spaces = ['LH401', 'LH501', 'LAB501', 'LAB502']

# schedule = {f"{slot}_{space}": Period(space, slot) for slot in slots for space in spaces}

# for key, value in sorted(schedule.items()):
#     print(f"{key}: {value}")
schedule = {slot: {space: None for space in spaces} for slot in slots}


## NSGA-II Documentation

In [6]:
# Code

In [ ]:
evaluate(schedule,groups_dict, lecturers_dict, activities_dict, spaces_dict,slots)

## MOEA/D Documentation

In [ ]:
# Code

In [ ]:
evaluate(schedule,groups_dict, lecturers_dict, activities_dict, spaces_dict,slots)

## SPEA2 Documentation

In [ ]:
# Code

In [ ]:
evaluate(schedule,groups_dict, lecturers_dict, activities_dict, spaces_dict,slots)